In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import math
import json
import nltk
import string
import re
import inflect
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.offline as py
import plotly.express as px
from plotly import tools
import plotly.figure_factory as ff
import warnings
from matplotlib import pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore")

#### Data Collection
<a class="anchor" id="dc"> </a>

In [ ]:
# importing all scraped datasets
df_food = pd.read_csv('food.csv')
df_sports = pd.read_csv('sports.csv')
df_health = pd.read_csv('health.csv')
df_business = pd.read_csv('business.csv')
df_entertainment = pd.read_csv('entertainment.csv')
df_environment = pd.read_csv('environment.csv')
df_politics = pd.read_csv('politics.csv')

#### Data Cleaning
<a class="anchor" id="dcl"> </a>

In [ ]:
# perform data cleaning on the food dataset
# display the food dataset
df_food
df_food.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,01-05-2022 00:00,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,01-05-2022 00:16,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,01-05-2022 00:25,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,01-05-2022 00:25,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,01-05-2022 00:53,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN


In [ ]:
# getting the info
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8487 entries, 0 to 8486
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TITLE                 8487 non-null   object 
 1   ARTICLE LINK          8487 non-null   object 
 2   PUBLISHED DATE (GMT)  8487 non-null   object 
 3   AUTHOR                2772 non-null   object 
 4   PUBLISHER             8487 non-null   object 
 5   COUNTRY               8487 non-null   object 
 6   CATEGORY              8487 non-null   object 
 7   LANGUAGE              8487 non-null   object 
 8   DESCRIPTION           8315 non-null   object 
 9   IMAGE URL             5526 non-null   object 
 10  VIDEO URL             0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 729.5+ KB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_food["DATETIME"] = pd.to_datetime(df_food["PUBLISHED DATE (GMT)"])
# display the dataframe
df_food.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,01-05-2022 00:00,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,01-05-2022 00:16,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN,2022-01-05 00:16:00
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,01-05-2022 00:25,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,01-05-2022 00:25,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,01-05-2022 00:53,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN,2022-01-05 00:53:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_food1 = df_food.drop('PUBLISHED DATE (GMT)', axis=1)
df_food1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN,2022-01-05 00:16:00
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN,2022-01-05 00:53:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_food1['DATE'] = df_food1['DATETIME'].dt.strftime('%d/%m/%Y')
df_food1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN,2022-01-05 00:16:00,05/01/2022
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00,05/01/2022
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00,05/01/2022
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN,2022-01-05 00:53:00,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_food1['GMT'] = df_food1['DATETIME'].dt.strftime('%H:%M')
df_food1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022,00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN,2022-01-05 00:16:00,05/01/2022,00:16
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00,05/01/2022,00:25
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,2022-01-05 00:25:00,05/01/2022,00:25
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN,2022-01-05 00:53:00,05/01/2022,00:53


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the DATETIME column
df_food2 = df_food1.drop('DATETIME', axis=1)
df_food2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,05/01/2022,00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,NaN,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,NaN,05/01/2022,00:16
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,NaN,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,05/01/2022,00:25
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,NaN,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,NaN,05/01/2022,00:25
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,NaN,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,NaN,05/01/2022,00:53


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_food2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE              0
ARTICLE LINK       0
AUTHOR          5715
PUBLISHER          0
COUNTRY            0
CATEGORY           0
LANGUAGE           0
DESCRIPTION      172
IMAGE URL       2961
VIDEO URL       8487
DATE               0
GMT                0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_food2['AUTHOR'] = df_food2['AUTHOR'].fillna('Unknown')
df_food2['IMAGE URL'] = df_food2['IMAGE URL'].fillna('Unknown')
df_food2['VIDEO URL'] = df_food2['VIDEO URL'].fillna('Unknown')
df_food2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,Unknown,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_food3 = df_food2.dropna()
df_food3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,Unknown,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
1,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16
2,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
3,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
4,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_food3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_food3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Health' 'Food' 'Entertainment' 'Environment' 'Top' 'Sports']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only food category
df_food4 = df_food3.loc[df_food3['CATEGORY'] == 'Food']
df_food4 = df_food4.reset_index(drop=True)
df_food4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16
1,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
2,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
3,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53
4,"""Viral Tiktok claims Sunnyboys were back at Wo...",https://www.dailymail.co.uk/femail/food/articl...,Unknown,Mail Online,United Kingdom,Food,English,Australian shoppers were sent into a spin afte...,https://i.dailymail.co.uk/1s/2022/05/01/02/572...,Unknown,05/01/2022,01:32


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_food4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_food5 = df_food4.reset_index(drop=True)
df_food5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16
1,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
2,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post ›,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
3,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53
4,"""Viral Tiktok claims Sunnyboys were back at Wo...",https://www.dailymail.co.uk/femail/food/articl...,Unknown,Mail Online,United Kingdom,Food,English,Australian shoppers were sent into a spin afte...,https://i.dailymail.co.uk/1s/2022/05/01/02/572...,Unknown,05/01/2022,01:32


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_food6 = df_food5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_food6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,A business duo has created an at-home hair kit...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16
1,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,"""Definitely not a need but I LOVE them, and th...",https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
2,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,Class is now in session.View Entire Post,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25
3,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,An ultra-chic contemporary home with a lap poo...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53
4,"""Viral Tiktok claims Sunnyboys were back at Wo...",https://www.dailymail.co.uk/femail/food/articl...,Unknown,Mail Online,United Kingdom,Food,English,Australian shoppers were sent into a spin afte...,https://i.dailymail.co.uk/1s/2022/05/01/02/572...,Unknown,05/01/2022,01:32


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_food6["DESCRIPTION"] = df_food6["DESCRIPTION"].apply(lower_text)
df_food6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       a business duo has created an at-home hair kit...
1       "definitely not a need but i love them, and th...
2               class is now in session.view entire post 
3       an ultra-chic contemporary home with a lap poo...
4       australian shoppers were sent into a spin afte...
                              ...                        
3751    comedian rob beckett hosts the six-part compet...
3752    among the cherished items in the new book are ...
3753    this week bel speaks to a woman who questions ...
3754    faith rempe, from midland, michigan, was known...
3755    that apple crisp isn't gonna make itself. view...
Name: DESCRIPTION, Length: 3756, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# removing the punctuations
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_food6["DESCRIPTION"] = df_food6["DESCRIPTION"].apply(remove_punct)
df_food6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       a business duo has created an at home hair kit...
1        definitely not a need but i love them  and th...
2               class is now in session view entire post 
3       an ultra chic contemporary home with a lap poo...
4       australian shoppers were sent into a spin afte...
                              ...                        
3751    comedian rob beckett hosts the six part compet...
3752    among the cherished items in the new book are ...
3753    this week bel speaks to a woman who questions ...
3754    faith rempe  from midland  michigan  was known...
3755    that apple crisp isn t gonna make itself  view...
Name: DESCRIPTION, Length: 3756, dtype: object

In [ ]:
# remove stopwords
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_food6["DESCRIPTION"] = df_food6["DESCRIPTION"].apply(remove_stop)
df_food6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       business duo created home hair kit bridges gap...
1       definitely need love make confident secure hap...
2                          class session view entire post
3       ultra chic contemporary home lap pool tennis c...
4       australian shoppers sent spin tiktok clip went...
                              ...                        
3751    comedian rob beckett hosts six part competitio...
3752    among cherished items new book electric go kar...
3753    week bel speaks woman questions keeps losing m...
3754    faith rempe midland michigan known faith gantn...
3755             apple crisp gon na make view entire post
Name: DESCRIPTION, Length: 3756, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_food6["DESCRIPTION"] = df_food6["DESCRIPTION"].apply(no_number)
df_food6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       business duo created home hair kit bridges gap...
1       definitely need love make confident secure hap...
2                          class session view entire post
3       ultra chic contemporary home lap pool tennis c...
4       australian shoppers sent spin tiktok clip went...
                              ...                        
3751    comedian rob beckett hosts six part competitio...
3752    among cherished items new book electric go kar...
3753    week bel speaks woman questions keeps losing m...
3754    faith rempe midland michigan known faith gantn...
3755             apple crisp gon na make view entire post
Name: DESCRIPTION, Length: 3756, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_food6["POSTAG"] = df_food6["DESCRIPTION"].apply(postag)
df_food6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0       [(business, NN), (duo, NN), (created, VBD), (h...
1       [(definitely, RB), (need, MD), (love, VB), (ma...
2       [(class, NN), (session, NN), (view, NN), (enti...
3       [(ultra, JJ), (chic, JJ), (contemporary, JJ), ...
4       [(australian, JJ), (shoppers, NNS), (sent, VBD...
                              ...                        
3751    [(comedian, JJ), (rob, NN), (beckett, NN), (ho...
3752    [(among, IN), (cherished, VBN), (items, NNS), ...
3753    [(week, NN), (bel, NN), (speaks, VBZ), (woman,...
3754    [(faith, NN), (rempe, NN), (midland, NN), (mic...
3755    [(apple, NN), (crisp, NN), (gon, NN), (na, TO)...
Name: POSTAG, Length: 3756, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_food6["DESCRIPTION"] = df_food6["DESCRIPTION"].apply(lemmatize_word)
df_food6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       business duo create home hair kit bridge gap b...
1       definitely need love make confident secure hap...
2                          class session view entire post
3       ultra chic contemporary home lap pool tennis c...
4       australian shoppers send spin tiktok clip go v...
                              ...                        
3751    comedian rob beckett host six part competition...
3752    among cherish items new book electric go kart ...
3753    week bel speak woman question keep lose men wa...
3754    faith rempe midland michigan know faith gantne...
3755             apple crisp gon na make view entire post
Name: DESCRIPTION, Length: 3756, dtype: object

In [ ]:
# perform data cleaning on the sports dataset
# display the sports dataset
df_sports.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,01-05-2022 00:00,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,01-05-2022 00:00,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN


In [ ]:
# getting the info
df_sports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475713 entries, 0 to 475712
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   TITLE                 475713 non-null  object
 1   ARTICLE LINK          475713 non-null  object
 2   PUBLISHED DATE (GMT)  475713 non-null  object
 3   AUTHOR                276743 non-null  object
 4   PUBLISHER             475713 non-null  object
 5   COUNTRY               475713 non-null  object
 6   CATEGORY              475713 non-null  object
 7   LANGUAGE              475713 non-null  object
 8   DESCRIPTION           357788 non-null  object
 9   IMAGE URL             185516 non-null  object
 10  VIDEO URL             424 non-null     object
dtypes: object(11)
memory usage: 39.9+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_sports["DATETIME"] = pd.to_datetime(df_sports["PUBLISHED DATE (GMT)"])
# display the dataframe
df_sports.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN,2022-01-05
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN,2022-01-05
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:00,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN,2022-01-05
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,01-05-2022 00:00,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN,2022-01-05
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,01-05-2022 00:00,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN,2022-01-05


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_sports1 = df_sports.drop('PUBLISHED DATE (GMT)', axis=1)
df_sports1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN,2022-01-05
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN,2022-01-05
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN,2022-01-05
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN,2022-01-05
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN,2022-01-05


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_sports1['DATE'] = df_sports1['DATETIME'].dt.strftime('%d/%m/%Y')
df_sports1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN,2022-01-05,05/01/2022
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN,2022-01-05,05/01/2022
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN,2022-01-05,05/01/2022
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN,2022-01-05,05/01/2022
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN,2022-01-05,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_sports1['GMT'] = df_sports1['DATETIME'].dt.strftime('%H:%M')
df_sports1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN,2022-01-05,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN,2022-01-05,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN,2022-01-05,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN,2022-01-05,05/01/2022,00:00
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN,2022-01-05,05/01/2022,00:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the DATETIME column
df_sports2 = df_sports1.drop('DATETIME', axis=1)
df_sports2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,NaN,NaN,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,NaN,NaN,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,NaN,NaN,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,NaN,NaN,05/01/2022,00:00
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,NaN,Yahoo! News,United Kingdom,Sports,English,NaN,NaN,NaN,05/01/2022,00:00


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_sports2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE                0
ARTICLE LINK         0
AUTHOR          198970
PUBLISHER            0
COUNTRY              0
CATEGORY             0
LANGUAGE             0
DESCRIPTION     117925
IMAGE URL       290197
VIDEO URL       475289
DATE                 0
GMT                  0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_sports2['AUTHOR'] = df_sports2['AUTHOR'].fillna('Unknown')
df_sports2['IMAGE URL'] = df_sports2['IMAGE URL'].fillna('Unknown')
df_sports2['VIDEO URL'] = df_sports2['VIDEO URL'].fillna('Unknown')
df_sports2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,Unknown,Unknown,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,Unknown,Unknown,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,Unknown,Unknown,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,Unknown,Unknown,05/01/2022,00:00
4,"""Após empate no Brasileirão, torcedores do Atl...",https://esportes.yahoo.com/noticias/ap%C3%B3s-...,Unknown,Yahoo! News,United Kingdom,Sports,English,NaN,Unknown,Unknown,05/01/2022,00:00


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_sports3 = df_sports2.dropna()
df_sports3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,Unknown,Unknown,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,Unknown,Unknown,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,Unknown,Unknown,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,Unknown,Unknown,05/01/2022,00:00
5,"""Bengals address depth needs on defense with D...",https://www.washingtontimes.com/news/2022/apr/...,Mitch Stacy,"The Washington Times , America’s Newspaper",United States Of America,Sports,English,The Cincinnati Bengals focused on defense in t...,https://twt-thumbs.washtimes.com/media/image/2...,Unknown,05/01/2022,00:01


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_sports3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_sports3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Sports' 'Top']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only sports category
df_sports4 = df_sports3.loc[df_sports3['CATEGORY'] == 'Sports']
df_sports4 = df_sports4.reset_index(drop=True)
df_sports4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,Unknown,Unknown,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,Unknown,Unknown,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,Unknown,Unknown,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,Unknown,Unknown,05/01/2022,00:00
4,"""Bengals address depth needs on defense with D...",https://www.washingtontimes.com/news/2022/apr/...,Mitch Stacy,"The Washington Times , America’s Newspaper",United States Of America,Sports,English,The Cincinnati Bengals focused on defense in t...,https://twt-thumbs.washtimes.com/media/image/2...,Unknown,05/01/2022,00:01


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_sports4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_sports5 = df_sports4.reset_index(drop=True)
df_sports5
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs’ draft...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs’ draft ...,Unknown,Unknown,05/01/2022,00:00
1,"""Jürgen Klopp may have let slip his Liverpool ...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Sports,English,Jürgen Klopp may have let slip his Liverpool s...,Unknown,Unknown,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,Unknown,Unknown,05/01/2022,00:00
3,"""“Clyde Drexler was a threat, but I took offen...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,Unknown,Unknown,05/01/2022,00:00
4,"""Bengals address depth needs on defense with D...",https://www.washingtontimes.com/news/2022/apr/...,Mitch Stacy,"The Washington Times , America’s Newspaper",United States Of America,Sports,English,The Cincinnati Bengals focused on defense in t...,https://twt-thumbs.washtimes.com/media/image/2...,Unknown,05/01/2022,00:01
...,...,...,...,...,...,...,...,...,...,...,...,...
285102,"""Rangers can leapfrog Celtic at top of the Sco...",https://www.belfasttelegraph.co.uk/sport/footb...,Unknown,Belfast Telegraph,United Kingdom,Sports,English,International breaks can be disruptive for clu...,https://www.belfasttelegraph.co.uk/sport/footb...,Unknown,30/09/2022,23:55
285103,"""Roll cage damage puts Evans out of WRC Rally ...",https://www.autosport.com/wrc/news/roll-cage-d...,Unknown,Autosport,United Kingdom,Sports,English,The Welshman started the day having inherited ...,https://cdn-1.motorsport.com/images/amp/YP3LQ4...,Unknown,30/09/2022,23:56
285104,"""The Raptors will give Scottie Barnes every sh...",https://www.thestar.com/sports/raptors/analysi...,Doug Smith - Sports Reporter,Toronto Star,Canada,Sports,English,The reason so many see such promise in the 21-...,Unknown,Unknown,30/09/2022,23:57
285105,"""Bringing up top prospect Francisco Alvarez wo...",https://nypost.com/2022/09/30/bringing-up-top-...,Joel Sherman,New York Post,United States Of America,Sports,English,The question at this point is why not. Not why.,https://nypost.com/wp-content/uploads/sites/2/...,Unknown,30/09/2022,23:57


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_sports6 = df_sports5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_sports6
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""6 winners and 4 losers from the Chiefs draft ...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,6 winners and 4 losers from the Chiefs draft w...,Unknown,Unknown,05/01/2022,00:00
1,"""Jrgen Klopp may have let slip his Liverpool s...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Sports,English,Jrgen Klopp may have let slip his Liverpool su...,Unknown,Unknown,05/01/2022,00:00
2,"""The Most HYPED Moments Of Round 1 | #NBAPlayo...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United States Of America,Sports,English,The Most HYPED Moments Of Round 1 | #NBAPlayof...,Unknown,Unknown,05/01/2022,00:00
3,"""Clyde Drexler was a threat, but I took offens...",https://thesportsrush.com/nba-news-clyde-drexl...,Arthur Sorcier,The Sportsrush,United States Of America,Sports,English,The 90s was an era that was simply dominated b...,Unknown,Unknown,05/01/2022,00:00
4,"""Bengals address depth needs on defense with D...",https://www.washingtontimes.com/news/2022/apr/...,Mitch Stacy,"The Washington Times , Americas Newspaper",United States Of America,Sports,English,The Cincinnati Bengals focused on defense in t...,https://twt-thumbs.washtimes.com/media/image/2...,Unknown,05/01/2022,00:01
...,...,...,...,...,...,...,...,...,...,...,...,...
285102,"""Rangers can leapfrog Celtic at top of the Sco...",https://www.belfasttelegraph.co.uk/sport/footb...,Unknown,Belfast Telegraph,United Kingdom,Sports,English,International breaks can be disruptive for clu...,https://www.belfasttelegraph.co.uk/sport/footb...,Unknown,30/09/2022,23:55
285103,"""Roll cage damage puts Evans out of WRC Rally ...",https://www.autosport.com/wrc/news/roll-cage-d...,Unknown,Autosport,United Kingdom,Sports,English,The Welshman started the day having inherited ...,https://cdn-1.motorsport.com/images/amp/YP3LQ4...,Unknown,30/09/2022,23:56
285104,"""The Raptors will give Scottie Barnes every sh...",https://www.thestar.com/sports/raptors/analysi...,Doug Smith - Sports Reporter,Toronto Star,Canada,Sports,English,The reason so many see such promise in the 21-...,Unknown,Unknown,30/09/2022,23:57
285105,"""Bringing up top prospect Francisco Alvarez wo...",https://nypost.com/2022/09/30/bringing-up-top-...,Joel Sherman,New York Post,United States Of America,Sports,English,The question at this point is why not. Not why.,https://nypost.com/wp-content/uploads/sites/2/...,Unknown,30/09/2022,23:57


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_sports6["DESCRIPTION"] = df_sports6["DESCRIPTION"].apply(lower_text)
df_sports6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         6 winners and 4 losers from the chiefs draft w...
1         jrgen klopp may have let slip his liverpool su...
2         the most hyped moments of round 1 | #nbaplayof...
3         the 90s was an era that was simply dominated b...
4         the cincinnati bengals focused on defense in t...
                                ...                        
285102    international breaks can be disruptive for clu...
285103    the welshman started the day having inherited ...
285104    the reason so many see such promise in the 21-...
285105      the question at this point is why not. not why.
285106    phil salt powers england to emphatic victory a...
Name: DESCRIPTION, Length: 285107, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# removing the punctuations
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_sports6["DESCRIPTION"] = df_sports6["DESCRIPTION"].apply(remove_punct)
df_sports6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         6 winners and 4 losers from the chiefs draft w...
1         jrgen klopp may have let slip his liverpool su...
2         the most hyped moments of round 1    nbaplayof...
3         the 90s was an era that was simply dominated b...
4         the cincinnati bengals focused on defense in t...
                                ...                        
285102    international breaks can be disruptive for clu...
285103    the welshman started the day having inherited ...
285104    the reason so many see such promise in the 21 ...
285105      the question at this point is why not  not why 
285106    phil salt powers england to emphatic victory a...
Name: DESCRIPTION, Length: 285107, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_sports6["DESCRIPTION"] = df_sports6["DESCRIPTION"].apply(remove_stop)
df_sports6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         6 winners 4 losers chiefs draft weekendarrowhe...
1         jrgen klopp may let slip liverpool successor i...
2         hyped moments round 1 nbaplayoffs presented go...
3         90s era simply dominated michael jordan howeve...
4         cincinnati bengals focused defense nfl draft u...
                                ...                        
285102    international breaks disruptive club managers ...
285103    welshman started day inherited lead hyundais o...
285104    reason many see promise 21 year old barnes bas...
285105                                       question point
285106    phil salt powers england emphatic victory leve...
Name: DESCRIPTION, Length: 285107, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_sports6["DESCRIPTION"] = df_sports6["DESCRIPTION"].apply(no_number)
df_sports6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0          winners  losers chiefs draft weekendarrowhead...
1         jrgen klopp may let slip liverpool successor i...
2         hyped moments round  nbaplayoffs presented goo...
3         s era simply dominated michael jordan however ...
4         cincinnati bengals focused defense nfl draft u...
                                ...                        
285102    international breaks disruptive club managers ...
285103    welshman started day inherited lead hyundais o...
285104    reason many see promise  year old barnes basel...
285105                                       question point
285106    phil salt powers england emphatic victory leve...
Name: DESCRIPTION, Length: 285107, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_sports6["POSTAG"] = df_sports6["DESCRIPTION"].apply(postag)
df_sports6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0         [(winners, NNS), (losers, NNS), (chiefs, VBP),...
1         [(jrgen, NN), (klopp, NN), (may, MD), (let, VB...
2         [(hyped, JJ), (moments, NNS), (round, JJ), (nb...
3         [(era, NN), (simply, RB), (dominated, VBD), (m...
4         [(cincinnati, NN), (bengals, NNS), (focused, V...
                                ...                        
285102    [(international, JJ), (breaks, NNS), (disrupti...
285103    [(welshman, NN), (started, VBD), (day, NN), (i...
285104    [(reason, NN), (many, JJ), (see, VBP), (promis...
285105                        [(question, NN), (point, NN)]
285106    [(phil, NN), (salt, NN), (powers, NNS), (engla...
Name: POSTAG, Length: 285107, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_sports6["DESCRIPTION"] = df_sports6["DESCRIPTION"].apply(lemmatize_word)
df_sports6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         winners losers chiefs draft weekendarrowhead p...
1         jrgen klopp may let slip liverpool successor i...
2         hype moments round nbaplayoffs present google ...
3         s era simply dominate michael jordan however c...
4         cincinnati bengals focus defense nfl draft use...
                                ...                        
285102    international break disruptive club managers o...
285103    welshman start day inherit lead hyundais ott t...
285104    reason many see promise year old barnes baseli...
285105                                       question point
285106    phil salt power england emphatic victory level...
Name: DESCRIPTION, Length: 285107, dtype: object

In [ ]:
# perform data cleaning on the health dataset
# display the health dataset
df_health.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,01-05-2022 00:00,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,01-05-2022 00:00,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:02,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,01-05-2022 00:17,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,01-05-2022 00:23,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN


In [ ]:
# getting the info
df_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71701 entries, 0 to 71700
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   TITLE                 71701 non-null  object
 1   ARTICLE LINK          71701 non-null  object
 2   PUBLISHED DATE (GMT)  71701 non-null  object
 3   AUTHOR                30066 non-null  object
 4   PUBLISHER             71701 non-null  object
 5   COUNTRY               71701 non-null  object
 6   CATEGORY              71701 non-null  object
 7   LANGUAGE              71701 non-null  object
 8   DESCRIPTION           60834 non-null  object
 9   IMAGE URL             31050 non-null  object
 10  VIDEO URL             420 non-null    object
dtypes: object(11)
memory usage: 6.0+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_health["DATETIME"] = pd.to_datetime(df_health["PUBLISHED DATE (GMT)"])
# display the dataframe
df_health.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,01-05-2022 00:00,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,01-05-2022 00:00,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,01-05-2022 00:02,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN,2022-01-05 00:02:00
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,01-05-2022 00:17,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:17:00
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,01-05-2022 00:23,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:23:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_health1 = df_health.drop('PUBLISHED DATE (GMT)', axis=1)
df_health1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN,2022-01-05 00:02:00
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:17:00
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:23:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_health1['DATE'] = df_health1['DATETIME'].dt.strftime('%d/%m/%Y')
df_health1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN,2022-01-05 00:02:00,05/01/2022
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:17:00,05/01/2022
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:23:00,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_health1['GMT'] = df_health1['DATETIME'].dt.strftime('%H:%M')
df_health1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022,00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,2022-01-05 00:00:00,05/01/2022,00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN,2022-01-05 00:02:00,05/01/2022,00:02
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:17:00,05/01/2022,00:17
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,2022-01-05 00:23:00,05/01/2022,00:23


In [ ]:
# drop the DATETIME column
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df_health2 = df_health1.drop('DATETIME', axis=1)
df_health2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,NaN,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,05/01/2022,00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,NaN,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,NaN,NaN,05/01/2022,00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,NaN,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,NaN,NaN,05/01/2022,00:02
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,NaN,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,05/01/2022,00:17
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,NaN,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,NaN,05/01/2022,00:23


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_health2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE               0
ARTICLE LINK        0
AUTHOR          41635
PUBLISHER           0
COUNTRY             0
CATEGORY            0
LANGUAGE            0
DESCRIPTION     10867
IMAGE URL       40651
VIDEO URL       71281
DATE                0
GMT                 0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_health2['AUTHOR'] = df_health2['AUTHOR'].fillna('Unknown')
df_health2['IMAGE URL'] = df_health2['IMAGE URL'].fillna('Unknown')
df_health2['VIDEO URL'] = df_health2['VIDEO URL'].fillna('Unknown')
df_health2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,Unknown,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,Unknown,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,Unknown,Unknown,05/01/2022,00:02
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,Unknown,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:17
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,Unknown,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:23


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_health3 = df_health2.dropna()
df_health3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,Unknown,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,Unknown,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,Unknown,Unknown,05/01/2022,00:02
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,Unknown,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:17
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,Unknown,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:23


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_health3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_health3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Health' 'Science' 'Technology' 'Top' 'Business' 'Entertainment' 'Sports']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only health category
df_health4 = df_health3.loc[df_health3['CATEGORY'] == 'Health']
df_health4 = df_health4.reset_index(drop=True)
df_health4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Nadia bares all for new campaign""",https://www.heraldsun.com.au/lifestyle/celebri...,Unknown,Herald Sun,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
1,"""Nadia bares all for new campaign""",https://www.cairnspost.com.au/lifestyle/celebr...,Unknown,Cairns Post,Australia,Health,English,Nadia Bartel has posed “naked” in a sizzling c...,Unknown,Unknown,05/01/2022,00:00
2,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,Unknown,Unknown,05/01/2022,00:02
3,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,Unknown,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:17
4,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,Unknown,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:23


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_health4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_health5 = df_health4.reset_index(drop=True)
df_health5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,Unknown,Unknown,05/01/2022,00:02
1,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,Unknown,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:17
2,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,Unknown,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:23
3,"""Oh Cum On, Why Are Men Having More Orgasms Th...",https://www.lifehacker.com.au/2022/05/oh-cum-o...,Nicole Andrejek,Lifehacker,Australia,Health,English,Sex researchers consistently find that men are...,https://imgix.lifehacker.com.au/content/upload...,Unknown,05/01/2022,00:30
4,"""Baby boy with acute hepatitis of unknown caus...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,Singapore,Health,English,Baby boy with acute hepatitis of unknown cause...,Unknown,Unknown,05/01/2022,00:39


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_health6 = df_health5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_health6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Ditching 'crazy' rules that prevent doctors p...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,United Kingdom,Health,English,Ditching 'crazy' rules that prevent doctors pr...,Unknown,Unknown,05/01/2022,00:02
1,"""University of Arizona, Oklahoma State Univers...",https://www.news-medical.net/news/20220430/Uni...,Unknown,Medical News,United States Of America,Health,English,"Robert C. Robbins, MD, president of the Univer...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:17
2,"""Trial shows the efficacy of anti-TNF drug for...",https://www.news-medical.net/news/20220430/Tri...,Unknown,Medical News,United States Of America,Health,English,"Researchers at the Kennedy Institute, Universi...",https://d2jx2rerrg6sh3.cloudfront.net/image-ha...,Unknown,05/01/2022,00:23
3,"""Oh Cum On, Why Are Men Having More Orgasms Th...",https://www.lifehacker.com.au/2022/05/oh-cum-o...,Nicole Andrejek,Lifehacker,Australia,Health,English,Sex researchers consistently find that men are...,https://imgix.lifehacker.com.au/content/upload...,Unknown,05/01/2022,00:30
4,"""Baby boy with acute hepatitis of unknown caus...",https://news.google.com/__i/rss/rd/articles/CB...,Unknown,Google News,Singapore,Health,English,Baby boy with acute hepatitis of unknown cause...,Unknown,Unknown,05/01/2022,00:39


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_health6["DESCRIPTION"] = df_health6["DESCRIPTION"].apply(lower_text)
df_health6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        ditching 'crazy' rules that prevent doctors pr...
1        robert c. robbins, md, president of the univer...
2        researchers at the kennedy institute, universi...
3        sex researchers consistently find that men are...
4        baby boy with acute hepatitis of unknown cause...
                               ...                        
40205    lytgobi (pronounced "light-goh-bee") delivered...
40206    waltham, mass., sept. 30, 2022 (globe newswire...
40207    the dolphins said he was conscious and had mov...
40208    scientists are still trying to understand why ...
40209    after 50 years of research, eminent stanford u...
Name: DESCRIPTION, Length: 40210, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# removing the punctuations
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_health6["DESCRIPTION"] = df_health6["DESCRIPTION"].apply(remove_punct)
df_health6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        ditching  crazy  rules that prevent doctors pr...
1        robert c  robbins  md  president of the univer...
2        researchers at the kennedy institute  universi...
3        sex researchers consistently find that men are...
4        baby boy with acute hepatitis of unknown cause...
                               ...                        
40205    lytgobi  pronounced  light goh bee   delivered...
40206    waltham  mass   sept  30  2022  globe newswire...
40207    the dolphins said he was conscious and had mov...
40208    scientists are still trying to understand why ...
40209    after 50 years of research  eminent stanford u...
Name: DESCRIPTION, Length: 40210, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_health6["DESCRIPTION"] = df_health6["DESCRIPTION"].apply(remove_stop)
df_health6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        ditching crazy rules prevent doctors prescribi...
1        robert c robbins md president university arizo...
2        researchers kennedy institute university oxfor...
3        sex researchers consistently find men far orga...
4        baby boy acute hepatitis unknown cause hospita...
                               ...                        
40205    lytgobi pronounced light goh bee delivered obj...
40206    waltham mass sept 30 2022 globe newswire apell...
40207    dolphins said conscious movement extremities d...
40208    scientists still trying understand many breast...
40209    50 years research eminent stanford university ...
Name: DESCRIPTION, Length: 40210, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_health6["DESCRIPTION"] = df_health6["DESCRIPTION"].apply(no_number)
df_health6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        ditching crazy rules prevent doctors prescribi...
1        robert c robbins md president university arizo...
2        researchers kennedy institute university oxfor...
3        sex researchers consistently find men far orga...
4        baby boy acute hepatitis unknown cause hospita...
                               ...                        
40205    lytgobi pronounced light goh bee delivered obj...
40206    waltham mass sept   globe newswire apellis pha...
40207    dolphins said conscious movement extremities d...
40208    scientists still trying understand many breast...
40209     years research eminent stanford university sl...
Name: DESCRIPTION, Length: 40210, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_health6["POSTAG"] = df_health6["DESCRIPTION"].apply(postag)
df_health6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0        [(ditching, VBG), (crazy, NN), (rules, NNS), (...
1        [(robert, JJ), (c, NN), (robbins, NNS), (md, V...
2        [(researchers, NNS), (kennedy, VBP), (institut...
3        [(sex, NN), (researchers, NNS), (consistently,...
4        [(baby, NN), (boy, VBD), (acute, JJ), (hepatit...
                               ...                        
40205    [(lytgobi, NN), (pronounced, VBD), (light, JJ)...
40206    [(waltham, JJ), (mass, NN), (sept, VBD), (glob...
40207    [(dolphins, NNS), (said, VBD), (conscious, JJ)...
40208    [(scientists, NNS), (still, RB), (trying, VBG)...
40209    [(years, NNS), (research, NN), (eminent, NN), ...
Name: POSTAG, Length: 40210, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_health6["DESCRIPTION"] = df_health6["DESCRIPTION"].apply(lemmatize_word)
df_health6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        ditch crazy rule prevent doctor prescribe hrt ...
1        robert c robbins md president university arizo...
2        researchers kennedy institute university oxfor...
3        sex researchers consistently find men far orga...
4        baby boy acute hepatitis unknown cause hospita...
                               ...                        
40205    lytgobi pronounce light goh bee deliver object...
40206    waltham mass sept globe newswire apellis pharm...
40207    dolphins say conscious movement extremities di...
40208    scientists still try understand many breast ca...
40209    years research eminent stanford university sle...
Name: DESCRIPTION, Length: 40210, dtype: object

In [ ]:
# perform data cleaning on the business dataset
# display the business dataset
df_business.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,01-05-2022 00:00,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,01-05-2022 00:00,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,01-05-2022 00:03,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,01-05-2022 00:04,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,01-05-2022 00:10,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN


In [ ]:
# getting the info
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283598 entries, 0 to 283597
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   TITLE                 283598 non-null  object
 1   ARTICLE LINK          283598 non-null  object
 2   PUBLISHED DATE (GMT)  283598 non-null  object
 3   AUTHOR                140681 non-null  object
 4   PUBLISHER             283154 non-null  object
 5   COUNTRY               283598 non-null  object
 6   CATEGORY              283598 non-null  object
 7   LANGUAGE              283598 non-null  object
 8   DESCRIPTION           234473 non-null  object
 9   IMAGE URL             131354 non-null  object
 10  VIDEO URL             84 non-null      object
dtypes: object(11)
memory usage: 23.8+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_business["DATETIME"] = pd.to_datetime(df_business["PUBLISHED DATE (GMT)"])
# display the dataframe
df_business.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,01-05-2022 00:00,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN,2022-01-05 00:00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,01-05-2022 00:00,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,01-05-2022 00:03,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:03:00
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,01-05-2022 00:04,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN,2022-01-05 00:04:00
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,01-05-2022 00:10,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN,2022-01-05 00:10:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_business1 = df_business.drop('PUBLISHED DATE (GMT)', axis=1)
df_business1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN,2022-01-05 00:00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:03:00
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN,2022-01-05 00:04:00
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN,2022-01-05 00:10:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_business1['DATE'] = df_business1['DATETIME'].dt.strftime('%d/%m/%Y')
df_business1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN,2022-01-05 00:00:00,05/01/2022
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:00:00,05/01/2022
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:03:00,05/01/2022
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN,2022-01-05 00:04:00,05/01/2022
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN,2022-01-05 00:10:00,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_business1['GMT'] = df_business1['DATETIME'].dt.strftime('%H:%M')
df_business1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN,2022-01-05 00:00:00,05/01/2022,00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:00:00,05/01/2022,00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,2022-01-05 00:03:00,05/01/2022,00:03
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN,2022-01-05 00:04:00,05/01/2022,00:04
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN,2022-01-05 00:10:00,05/01/2022,00:10


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the DATETIME column
df_business2 = df_business1.drop('DATETIME', axis=1)
df_business2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,NaN,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,NaN,NaN,05/01/2022,00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,05/01/2022,00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,NaN,05/01/2022,00:03
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,NaN,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,NaN,NaN,05/01/2022,00:04
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,NaN,The Times Of India,India,Business,English,NaN,NaN,NaN,05/01/2022,00:10


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_business2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE                0
ARTICLE LINK         0
AUTHOR          142917
PUBLISHER          444
COUNTRY              0
CATEGORY             0
LANGUAGE             0
DESCRIPTION      49125
IMAGE URL       152244
VIDEO URL       283514
DATE                 0
GMT                  0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_business2['AUTHOR'] = df_business2['AUTHOR'].fillna('Unknown')
df_business2['IMAGE URL'] = df_business2['IMAGE URL'].fillna('Unknown')
df_business2['VIDEO URL'] = df_business2['VIDEO URL'].fillna('Unknown')
df_business2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,Unknown,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,Unknown,Unknown,05/01/2022,00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:03
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,Unknown,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,Unknown,Unknown,05/01/2022,00:04
4,"""Railways' daily coal loading up 20% over last...",https://timesofindia.indiatimes.com/business/i...,Unknown,The Times Of India,India,Business,English,NaN,Unknown,Unknown,05/01/2022,00:10


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_business3 = df_business2.dropna()
df_business3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,Unknown,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,Unknown,Unknown,05/01/2022,00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:03
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,Unknown,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,Unknown,Unknown,05/01/2022,00:04
5,"""3 Key Warren Buffett Takeaways on the State o...",https://www.fool.com/investing/2022/04/30/3-ke...,newsfeedback@fool.com (Daniel Foelber),Fool,Canada,Business,English,"Uncertainty is high, but that doesn't mean it'...",Unknown,Unknown,05/01/2022,00:13


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_business3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_business3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Business' 'Top' 'Technology']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only business category
df_business4 = df_business3.loc[df_business3['CATEGORY'] == 'Business']
df_business4 = df_business4.reset_index(drop=True)
df_business4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Fed’s balance-sheet is about to shrink. W...",https://www.economist.com/finance-and-economic...,Unknown,The Economist,United Kingdom,Business,English,Could the giant market for Treasury bonds seiz...,Unknown,Unknown,05/01/2022,00:00
1,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:00
2,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:03
3,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,Unknown,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,Unknown,Unknown,05/01/2022,00:04
4,"""3 Key Warren Buffett Takeaways on the State o...",https://www.fool.com/investing/2022/04/30/3-ke...,newsfeedback@fool.com (Daniel Foelber),Fool,Canada,Business,English,"Uncertainty is high, but that doesn't mean it'...",Unknown,Unknown,05/01/2022,00:13


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_business4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_business5 = df_business4.reset_index(drop=True)
df_business5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:00
1,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:03
2,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,Unknown,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,Unknown,Unknown,05/01/2022,00:04
3,"""3 Key Warren Buffett Takeaways on the State o...",https://www.fool.com/investing/2022/04/30/3-ke...,newsfeedback@fool.com (Daniel Foelber),Fool,Canada,Business,English,"Uncertainty is high, but that doesn't mean it'...",Unknown,Unknown,05/01/2022,00:13
4,"""S. Korea’s Exports to China Drop Amid Fallout...",https://www.bloombergquint.com/global-economic...,Sam Kim,Bq Prime,India,Business,English,South Korea’s Exports to China Drop Amid Fallo...,Unknown,Unknown,05/01/2022,00:21


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_business6 = df_business5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_business6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""The Good, Bad And Ugly From The Green Bay Pac...",https://www.forbes.com/sites/robreischel/2022/...,"Rob Reischel, Contributor",Forbes,United States Of America,Business,English,Green Bay Packers general manager Brian Guteku...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:00
1,"""Indianapolis Colts Go All-In On Athleticism D...",https://www.forbes.com/sites/evansidery/2022/0...,"Evan Sidery, Contributor",Forbes,United States Of America,Business,English,The Indianapolis Colts made history during the...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,Unknown,05/01/2022,00:03
2,""": At the Berkshire Hathaway annual meeting, W...",http://www.marketwatch.com/news/story.asp?guid...,Unknown,Marketwatch,Philippines,Business,English,At the start of the Berkshire Hathaway BRK.B B...,Unknown,Unknown,05/01/2022,00:04
3,"""3 Key Warren Buffett Takeaways on the State o...",https://www.fool.com/investing/2022/04/30/3-ke...,newsfeedback@fool.com (Daniel Foelber),Fool,Canada,Business,English,"Uncertainty is high, but that doesn't mean it'...",Unknown,Unknown,05/01/2022,00:13
4,"""S. Koreas Exports to China Drop Amid Fallout ...",https://www.bloombergquint.com/global-economic...,Sam Kim,Bq Prime,India,Business,English,South Koreas Exports to China Drop Amid Fallou...,Unknown,Unknown,05/01/2022,00:21


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_business6["DESCRIPTION"] = df_business6["DESCRIPTION"].apply(lower_text)
df_business6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         green bay packers general manager brian guteku...
1         the indianapolis colts made history during the...
2         at the start of the berkshire hathaway brk.b b...
3         uncertainty is high, but that doesn't mean it'...
4         south koreas exports to china drop amid fallou...
                                ...                        
184189    petroleum producers are crafting their spendin...
184190    wall street posts third straight quarterly los...
184191    handmade and one of a kind, each piece by ksm ...
184192    growth in the core sector, comprising eight in...
184193    the regulator also asked companies, including ...
Name: DESCRIPTION, Length: 184194, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# removing the punctuations
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_business6["DESCRIPTION"] = df_business6["DESCRIPTION"].apply(remove_punct)
df_business6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         green bay packers general manager brian guteku...
1         the indianapolis colts made history during the...
2         at the start of the berkshire hathaway brk b b...
3         uncertainty is high  but that doesn t mean it ...
4         south koreas exports to china drop amid fallou...
                                ...                        
184189    petroleum producers are crafting their spendin...
184190    wall street posts third straight quarterly los...
184191    handmade and one of a kind  each piece by ksm ...
184192    growth in the core sector  comprising eight in...
184193    the regulator also asked companies  including ...
Name: DESCRIPTION, Length: 184194, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_business6["DESCRIPTION"] = df_business6["DESCRIPTION"].apply(remove_stop)
df_business6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         green bay packers general manager brian guteku...
1         indianapolis colts made history 2022 nfl draft...
2         start berkshire hathaway brk b brk annual meet...
3                          uncertainty high mean time panic
4         south koreas exports china drop amid fallout l...
                                ...                        
184189    petroleum producers crafting spending plans co...
184190    wall street posts third straight quarterly los...
184191    handmade one kind piece ksm handcrafted creati...
184192    growth core sector comprising eight infrastruc...
184193    regulator also asked companies including new a...
Name: DESCRIPTION, Length: 184194, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_business6["DESCRIPTION"] = df_business6["DESCRIPTION"].apply(no_number)
df_business6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         green bay packers general manager brian guteku...
1         indianapolis colts made history  nfl draft ans...
2         start berkshire hathaway brk b brk annual meet...
3                          uncertainty high mean time panic
4         south koreas exports china drop amid fallout l...
                                ...                        
184189    petroleum producers crafting spending plans co...
184190    wall street posts third straight quarterly los...
184191    handmade one kind piece ksm handcrafted creati...
184192    growth core sector comprising eight infrastruc...
184193    regulator also asked companies including new a...
Name: DESCRIPTION, Length: 184194, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_business6["POSTAG"] = df_business6["DESCRIPTION"].apply(postag)
df_business6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0         [(green, JJ), (bay, NN), (packers, NNS), (gene...
1         [(indianapolis, NN), (colts, NNS), (made, VBD)...
2         [(start, NN), (berkshire, VB), (hathaway, NN),...
3         [(uncertainty, NN), (high, JJ), (mean, JJ), (t...
4         [(south, JJ), (koreas, NNS), (exports, NNS), (...
                                ...                        
184189    [(petroleum, NN), (producers, NNS), (crafting,...
184190    [(wall, JJ), (street, NN), (posts, NNS), (thir...
184191    [(handmade, VB), (one, CD), (kind, NN), (piece...
184192    [(growth, NN), (core, NN), (sector, NN), (comp...
184193    [(regulator, NN), (also, RB), (asked, VBD), (c...
Name: POSTAG, Length: 184194, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_business6["DESCRIPTION"] = df_business6["DESCRIPTION"].apply(lemmatize_word)
df_business6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         green bay packers general manager brian guteku...
1         indianapolis colts make history nfl draft answ...
2         start berkshire hathaway brk b brk annual meet...
3                          uncertainty high mean time panic
4         south koreas export china drop amid fallout lo...
                                ...                        
184189    petroleum producers craft spend plan come year...
184190    wall street post third straight quarterly loss...
184191    handmade one kind piece ksm handcraft creation...
184192    growth core sector comprise eight infrastructu...
184193    regulator also ask company include new age tec...
Name: DESCRIPTION, Length: 184194, dtype: object

In [ ]:
# perform data cleaning on the entertainment dataset
# display the entertainment dataset
df_entertainment.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,01-05-2022 00:01,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,01-05-2022 00:01,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,01-05-2022 00:02,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,01-05-2022 00:04,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,01-05-2022 00:05,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN


In [ ]:
# getting the info
df_entertainment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306728 entries, 0 to 306727
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   TITLE                 306728 non-null  object
 1   ARTICLE LINK          306728 non-null  object
 2   PUBLISHED DATE (GMT)  306728 non-null  object
 3   AUTHOR                187673 non-null  object
 4   PUBLISHER             306352 non-null  object
 5   COUNTRY               306728 non-null  object
 6   CATEGORY              306728 non-null  object
 7   LANGUAGE              306728 non-null  object
 8   DESCRIPTION           288140 non-null  object
 9   IMAGE URL             127567 non-null  object
 10  VIDEO URL             777 non-null     object
dtypes: object(11)
memory usage: 25.7+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_entertainment["DATETIME"] = pd.to_datetime(df_entertainment["PUBLISHED DATE (GMT)"])
# display the dataframe
df_entertainment.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,01-05-2022 00:01,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN,2022-01-05 00:01:00
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,01-05-2022 00:01,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN,2022-01-05 00:01:00
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,01-05-2022 00:02,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN,2022-01-05 00:02:00
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,01-05-2022 00:04,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN,2022-01-05 00:04:00
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,01-05-2022 00:05,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN,2022-01-05 00:05:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_entertainment1 = df_entertainment.drop('PUBLISHED DATE (GMT)', axis=1)
df_entertainment1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN,2022-01-05 00:01:00
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN,2022-01-05 00:01:00
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN,2022-01-05 00:02:00
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN,2022-01-05 00:04:00
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN,2022-01-05 00:05:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_entertainment1['DATE'] = df_entertainment1['DATETIME'].dt.strftime('%d/%m/%Y')
df_entertainment1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN,2022-01-05 00:01:00,05/01/2022
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN,2022-01-05 00:01:00,05/01/2022
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN,2022-01-05 00:02:00,05/01/2022
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN,2022-01-05 00:04:00,05/01/2022
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN,2022-01-05 00:05:00,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_entertainment1['GMT'] = df_entertainment1['DATETIME'].dt.strftime('%H:%M')
df_entertainment1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN,2022-01-05 00:01:00,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN,2022-01-05 00:01:00,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN,2022-01-05 00:02:00,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN,2022-01-05 00:04:00,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN,2022-01-05 00:05:00,05/01/2022,00:05


In [ ]:
# drop the DATETIME column
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df_entertainment2 = df_entertainment1.drop('DATETIME', axis=1)
df_entertainment2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,NaN,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,NaN,NaN,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,NaN,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,NaN,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,NaN,NaN,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,NaN,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,NaN,05/01/2022,00:05


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_entertainment2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE                0
ARTICLE LINK         0
AUTHOR          119055
PUBLISHER          376
COUNTRY              0
CATEGORY             0
LANGUAGE             0
DESCRIPTION      18588
IMAGE URL       179161
VIDEO URL       305951
DATE                 0
GMT                  0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_entertainment2['AUTHOR'] = df_entertainment2['AUTHOR'].fillna('Unknown')
df_entertainment2['IMAGE URL'] = df_entertainment2['IMAGE URL'].fillna('Unknown')
df_entertainment2['VIDEO URL'] = df_entertainment2['VIDEO URL'].fillna('Unknown')
df_entertainment2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,Unknown,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,Unknown,Unknown,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,Unknown,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,Unknown,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,Unknown,Unknown,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,Unknown,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,Unknown,05/01/2022,00:05


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_entertainment3 = df_entertainment2.dropna()
df_entertainment3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,Unknown,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,Unknown,Unknown,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,Unknown,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,Unknown,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,Unknown,Unknown,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,Unknown,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,Unknown,05/01/2022,00:05


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_entertainment3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_entertainment3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Entertainment' 'Top' 'Technology' 'Business' 'Sports']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only entertainment category
df_entertainment4 = df_entertainment3.loc[df_entertainment3['CATEGORY'] == 'Entertainment']
df_entertainment4 = df_entertainment4.reset_index(drop=True)
df_entertainment4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,Unknown,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,Unknown,Unknown,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,Unknown,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,Unknown,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,Unknown,Unknown,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,Unknown,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,Unknown,05/01/2022,00:05


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_entertainment4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_entertainment5 = df_entertainment4.reset_index(drop=True)
df_entertainment5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,Unknown,Cairns Post,Australia,Entertainment,English,The TV reunion we’ve been waiting for looks se...,Unknown,Unknown,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,Unknown,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,Unknown,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,Unknown,Unknown,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,Unknown,05/01/2022,00:04
4,"""PS Plus is ripping fans off before it’s even ...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sony’s attempts to...,https://metro.co.uk/wp-content/uploads/2022/03...,Unknown,05/01/2022,00:05


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_entertainment6 = df_entertainment5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_entertainment6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Bombshell ending for Neighbours""",https://www.cairnspost.com.au/entertainment/te...,Unknown,Cairns Post,Australia,Entertainment,English,The TV reunion weve been waiting for looks set...,Unknown,Unknown,05/01/2022,00:01
1,"""Dragon Ball Gets Manga Makeover From Toriko C...",https://comicbook.com/anime/news/dragon-ball-m...,Nick Valdez,Comic Book,United States Of America,Entertainment,English,Dragon Ball's manga has gotten a slick new mak...,https://sportshub.cbsistatic.com/i/2022/04/30/...,Unknown,05/01/2022,00:01
2,"""Mother reveals crazy way she caught husband c...",https://www.brisbanetimes.com.au/lifestyle/mot...,Unknown,Brisbane Times,Australia,Entertainment,English,TikToker explains how she caught her husband a...,Unknown,Unknown,05/01/2022,00:02
3,"""Impact Wrestling's Deonna Purrazzo Talks Unde...",https://comicbook.com/wwe/news/impact-wrestlin...,Matthew Aguilar,Comic Book,United States Of America,Entertainment,English,Things aren't slowing down in the least for Im...,https://sportshub.cbsistatic.com/i/2022/05/01/...,Unknown,05/01/2022,00:04
4,"""PS Plus is ripping fans off before its even o...",https://metro.co.uk/2022/05/01/ps-plus-is-ripp...,GameCentral,Metro,United Kingdom,Entertainment,English,A reader is not happy about Sonys attempts to ...,https://metro.co.uk/wp-content/uploads/2022/03...,Unknown,05/01/2022,00:05


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_entertainment6["DESCRIPTION"] = df_entertainment6["DESCRIPTION"].apply(lower_text)
df_entertainment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         the tv reunion weve been waiting for looks set...
1         dragon ball's manga has gotten a slick new mak...
2         tiktoker explains how she caught her husband a...
3         things aren't slowing down in the least for im...
4         a reader is not happy about sonys attempts to ...
                                ...                        
264821    series creator dan harmon and six of its nine ...
264822    akon, a famous american singer, has hailed wiz...
264823    dungeons & dragons is testing a fix to two-wea...
264824    pop veteran haddaway was due to perform with l...
264825    reese witherspoon is out of touch with the sla...
Name: DESCRIPTION, Length: 264826, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
# removing the punctuations
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_entertainment6["DESCRIPTION"] = df_entertainment6["DESCRIPTION"].apply(remove_punct)
df_entertainment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0         the tv reunion weve been waiting for looks set...
1         dragon ball s manga has gotten a slick new mak...
2         tiktoker explains how she caught her husband a...
3         things aren t slowing down in the least for im...
4         a reader is not happy about sonys attempts to ...
                                ...                        
264821    series creator dan harmon and six of its nine ...
264822    akon  a famous american singer  has hailed wiz...
264823    dungeons   dragons is testing a fix to two wea...
264824    pop veteran haddaway was due to perform with l...
264825    reese witherspoon is out of touch with the sla...
Name: DESCRIPTION, Length: 264826, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_entertainment6["DESCRIPTION"] = df_entertainment6["DESCRIPTION"].apply(remove_stop)
df_entertainment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0          tv reunion weve waiting looks set finally happen
1         dragon ball manga gotten slick new makeover se...
2         tiktoker explains caught husband nanny cheatin...
3         things slowing least impact wrestling deonna p...
4         reader happy sonys attempts block stacking ps ...
                                ...                        
264821    series creator dan harmon six nine original st...
264822    akon famous american singer hailed wizkid davi...
264823    dungeons dragons testing fix two weapon fighti...
264824    pop veteran haddaway due perform late rapper c...
264825            reese witherspoon touch slang kids saying
Name: DESCRIPTION, Length: 264826, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_entertainment6["DESCRIPTION"] = df_entertainment6["DESCRIPTION"].apply(no_number)
df_entertainment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0          tv reunion weve waiting looks set finally happen
1         dragon ball manga gotten slick new makeover se...
2         tiktoker explains caught husband nanny cheatin...
3         things slowing least impact wrestling deonna p...
4         reader happy sonys attempts block stacking ps ...
                                ...                        
264821    series creator dan harmon six nine original st...
264822    akon famous american singer hailed wizkid davi...
264823    dungeons dragons testing fix two weapon fighti...
264824    pop veteran haddaway due perform late rapper c...
264825            reese witherspoon touch slang kids saying
Name: DESCRIPTION, Length: 264826, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_entertainment6["POSTAG"] = df_entertainment6["DESCRIPTION"].apply(postag)
df_entertainment6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0         [(tv, NN), (reunion, NN), (weve, VBP), (waitin...
1         [(dragon, NN), (ball, NN), (manga, NN), (gotte...
2         [(tiktoker, NN), (explains, VBZ), (caught, JJ)...
3         [(things, NNS), (slowing, VBG), (least, JJS), ...
4         [(reader, NN), (happy, JJ), (sonys, NN), (atte...
                                ...                        
264821    [(series, NN), (creator, NN), (dan, JJ), (harm...
264822    [(akon, RB), (famous, JJ), (american, JJ), (si...
264823    [(dungeons, NNS), (dragons, NNS), (testing, VB...
264824    [(pop, NN), (veteran, NN), (haddaway, NN), (du...
264825    [(reese, JJ), (witherspoon, NN), (touch, NN), ...
Name: POSTAG, Length: 264826, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_entertainment6["DESCRIPTION"] = df_entertainment6["DESCRIPTION"].apply(lemmatize_word)
df_entertainment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0              tv reunion weve wait look set finally happen
1         dragon ball manga get slick new makeover serie...
2         tiktoker explain catch husband nanny cheat stu...
3         things slow least impact wrestle deonna purraz...
4         reader happy sonys attempt block stack ps subs...
                                ...                        
264821    series creator dan harmon six nine original st...
264822    akon famous american singer hail wizkid davido...
264823    dungeons dragons test fix two weapon fight pro...
264824    pop veteran haddaway due perform late rapper c...
264825                reese witherspoon touch slang kid say
Name: DESCRIPTION, Length: 264826, dtype: object

In [ ]:
# perform data cleaning on the environment dataset
# display the environment dataset
df_environment.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,01-05-2022 03:35,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,01-05-2022 03:55,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,01-05-2022 03:55,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,01-05-2022 05:51,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,01-05-2022 06:43,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN


In [ ]:
# getting the info
df_environment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15917 entries, 0 to 15916
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TITLE                 15917 non-null  object 
 1   ARTICLE LINK          15917 non-null  object 
 2   PUBLISHED DATE (GMT)  15917 non-null  object 
 3   AUTHOR                7072 non-null   object 
 4   PUBLISHER             15917 non-null  object 
 5   COUNTRY               15917 non-null  object 
 6   CATEGORY              15917 non-null  object 
 7   LANGUAGE              15917 non-null  object 
 8   DESCRIPTION           15749 non-null  object 
 9   IMAGE URL             0 non-null      float64
 10  VIDEO URL             0 non-null      float64
dtypes: float64(2), object(9)
memory usage: 1.3+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_environment["DATETIME"] = pd.to_datetime(df_environment["PUBLISHED DATE (GMT)"])
# display the dataframe
df_environment.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,01-05-2022 03:35,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN,2022-01-05 03:35:00
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,01-05-2022 03:55,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN,2022-01-05 03:55:00
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,01-05-2022 03:55,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN,2022-01-05 03:55:00
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,01-05-2022 05:51,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN,2022-01-05 05:51:00
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,01-05-2022 06:43,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN,2022-01-05 06:43:00


In [ ]:
# drop the PUBLISHED DATE (GMT) column
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df_environment1 = df_environment.drop('PUBLISHED DATE (GMT)', axis=1)
df_environment1.head()

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN,2022-01-05 03:35:00
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN,2022-01-05 03:55:00
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN,2022-01-05 03:55:00
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN,2022-01-05 05:51:00
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN,2022-01-05 06:43:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_environment1['DATE'] = df_environment1['DATETIME'].dt.strftime('%d/%m/%Y')
df_environment1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN,2022-01-05 03:35:00,05/01/2022
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN,2022-01-05 03:55:00,05/01/2022
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN,2022-01-05 03:55:00,05/01/2022
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN,2022-01-05 05:51:00,05/01/2022
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN,2022-01-05 06:43:00,05/01/2022


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# separating time from datetime
df_environment1['GMT'] = df_environment1['DATETIME'].dt.strftime('%H:%M')
df_environment1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN,2022-01-05 03:35:00,05/01/2022,03:35
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN,2022-01-05 03:55:00,05/01/2022,03:55
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN,2022-01-05 03:55:00,05/01/2022,03:55
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN,2022-01-05 05:51:00,05/01/2022,05:51
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN,2022-01-05 06:43:00,05/01/2022,06:43


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the DATETIME column
df_environment2 = df_environment1.drop('DATETIME', axis=1)
df_environment2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,NaN,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,NaN,NaN,05/01/2022,03:35
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,NaN,NaN,05/01/2022,03:55
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,NaN,NaN,05/01/2022,03:55
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,NaN,NaN,05/01/2022,05:51
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,NaN,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,NaN,NaN,05/01/2022,06:43


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_environment2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE               0
ARTICLE LINK        0
AUTHOR           8845
PUBLISHER           0
COUNTRY             0
CATEGORY            0
LANGUAGE            0
DESCRIPTION       168
IMAGE URL       15917
VIDEO URL       15917
DATE                0
GMT                 0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_environment2['AUTHOR'] = df_environment2['AUTHOR'].fillna('Unknown')
df_environment2['IMAGE URL'] = df_environment2['IMAGE URL'].fillna('Unknown')
df_environment2['VIDEO URL'] = df_environment2['VIDEO URL'].fillna('Unknown')
df_environment2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,Unknown,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,Unknown,Unknown,05/01/2022,03:35
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,Unknown,Unknown,05/01/2022,03:55
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,Unknown,Unknown,05/01/2022,03:55
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,Unknown,Unknown,05/01/2022,05:51
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,Unknown,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,Unknown,Unknown,05/01/2022,06:43


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_environment3 = df_environment2.dropna()
df_environment3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,Unknown,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,Unknown,Unknown,05/01/2022,03:35
1,"""Women who underwent abortions face ""trauma"" a...",https://www.scotsman.com/news/people/women-who...,Hannah Brown,The Scotsman,United Kingdom,Business,English,Women who went through abortions have told how...,Unknown,Unknown,05/01/2022,03:55
2,"""IndyRef2 will include new issues, says First ...",https://www.scotsman.com/news/politics/indyref...,Ilona Amos,The Scotsman,United Kingdom,Business,English,Nicola Sturgeon believes the next referendum t...,Unknown,Unknown,05/01/2022,03:55
3,"""Live updates l Explosion damages railway brid...",https://www.washingtontimes.com/news/2022/may/...,The Associated Press,"The Washington Times , America’s Newspaper",United States Of America,Technology,English,An explosive device damaged a railway bridge S...,Unknown,Unknown,05/01/2022,05:51
4,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,Unknown,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,Unknown,Unknown,05/01/2022,06:43


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_environment3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_environment3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Environment' 'Business' 'Technology' 'Entertainment' 'Top' 'World'
 'Health']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only environment category
df_environment4 = df_environment3.loc[df_environment3['CATEGORY'] == 'Environment']
df_environment4 = df_environment4.reset_index(drop=True)
df_environment4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,Unknown,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,Unknown,Unknown,05/01/2022,03:35
1,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,Unknown,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,Unknown,Unknown,05/01/2022,06:43
2,"""Local elections 2022: Greens winning hearts i...",https://www.independent.co.uk/news/uk/politics...,Colin Drury,Independent,United Kingdom,Environment,English,Campaigners welcomed on doorstep in Labour str...,Unknown,Unknown,05/01/2022,06:58
3,"""Halfmens: Four succulent poachers jailed for ...",https://mg.co.za/environment/2022-05-01-halfme...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,‘Worst case in history of Richtersveld Nationa...,Unknown,Unknown,05/01/2022,07:00
4,"""High court suspends environment department’s ...",https://mg.co.za/environment/2022-05-01-high-c...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,Interim interdict ‘spares death’ of 170 animal...,Unknown,Unknown,05/01/2022,07:00


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_environment4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_environment5 = df_environment4.reset_index(drop=True)
df_environment5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,Unknown,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,Unknown,Unknown,05/01/2022,03:35
1,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,Unknown,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,Unknown,Unknown,05/01/2022,06:43
2,"""Local elections 2022: Greens winning hearts i...",https://www.independent.co.uk/news/uk/politics...,Colin Drury,Independent,United Kingdom,Environment,English,Campaigners welcomed on doorstep in Labour str...,Unknown,Unknown,05/01/2022,06:58
3,"""Halfmens: Four succulent poachers jailed for ...",https://mg.co.za/environment/2022-05-01-halfme...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,‘Worst case in history of Richtersveld Nationa...,Unknown,Unknown,05/01/2022,07:00
4,"""High court suspends environment department’s ...",https://mg.co.za/environment/2022-05-01-high-c...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,Interim interdict ‘spares death’ of 170 animal...,Unknown,Unknown,05/01/2022,07:00


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_environment6 = df_environment5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_environment6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""First Alert Forecast: CBS2 4/30 Nightly Weath...",https://www.cbsnews.com/newyork/video/first-al...,Unknown,Newyork Cbs Local,United States Of America,Environment,English,CBS2's Vanessa Murdock has your weather foreca...,Unknown,Unknown,05/01/2022,03:35
1,"""BBC Weather: Brits to bask in 20C warmth as A...",https://www.express.co.uk/news/weather/1603744...,Unknown,Express,United Kingdom,Environment,English,BBC Weather forecaster Louise Lear said warmth...,Unknown,Unknown,05/01/2022,06:43
2,"""Local elections 2022: Greens winning hearts i...",https://www.independent.co.uk/news/uk/politics...,Colin Drury,Independent,United Kingdom,Environment,English,Campaigners welcomed on doorstep in Labour str...,Unknown,Unknown,05/01/2022,06:58
3,"""Halfmens: Four succulent poachers jailed for ...",https://mg.co.za/environment/2022-05-01-halfme...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,Worst case in history of Richtersveld National...,Unknown,Unknown,05/01/2022,07:00
4,"""High court suspends environment departments t...",https://mg.co.za/environment/2022-05-01-high-c...,Sheree Bega,Mail & Guardian,South Africa,Environment,English,Interim interdict spares death of 170 animals ...,Unknown,Unknown,05/01/2022,07:00


In [ ]:
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
# converting the text into lower case
def lower_text(text):
    text = text.lower()
    return text
df_environment6["DESCRIPTION"] = df_environment6["DESCRIPTION"].apply(lower_text)
df_environment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       cbs2's vanessa murdock has your weather foreca...
1       bbc weather forecaster louise lear said warmth...
2       campaigners welcomed on doorstep in labour str...
3       worst case in history of richtersveld national...
4       interim interdict spares death of 170 animals ...
                              ...                        
7276    two days after hurricane ian devastated the st...
7277          weather will be comfortable for fair goers.
7278    cbs2's john elliott has your first alert forec...
7279    urgent action is required to support the reint...
7280             sunshine and fried food in the forecast!
Name: DESCRIPTION, Length: 7281, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
# removing the punctuations
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_environment6["DESCRIPTION"] = df_environment6["DESCRIPTION"].apply(remove_punct)
df_environment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       cbs2 s vanessa murdock has your weather foreca...
1       bbc weather forecaster louise lear said warmth...
2       campaigners welcomed on doorstep in labour str...
3       worst case in history of richtersveld national...
4       interim interdict spares death of 170 animals ...
                              ...                        
7276    two days after hurricane ian devastated the st...
7277          weather will be comfortable for fair goers 
7278    cbs2 s john elliott has your first alert forec...
7279    urgent action is required to support the reint...
7280             sunshine and fried food in the forecast 
Name: DESCRIPTION, Length: 7281, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_environment6["DESCRIPTION"] = df_environment6["DESCRIPTION"].apply(remove_stop)
df_environment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0       cbs2 vanessa murdock weather forecast april 30...
1       bbc weather forecaster louise lear said warmth...
2       campaigners welcomed doorstep labour strong ho...
3       worst case history richtersveld national park ...
4       interim interdict spares death 170 animals han...
                              ...                        
7276    two days hurricane ian devastated state florid...
7277                       weather comfortable fair goers
7278    cbs2 john elliott first alert forecast septemb...
7279    urgent action required support reintroduction ...
7280                         sunshine fried food forecast
Name: DESCRIPTION, Length: 7281, dtype: object

In [ ]:
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
# remove the numbers and words containing number
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_environment6["DESCRIPTION"] = df_environment6["DESCRIPTION"].apply(no_number)
df_environment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0          cbs vanessa murdock weather forecast april   p
1       bbc weather forecaster louise lear said warmth...
2       campaigners welcomed doorstep labour strong ho...
3       worst case history richtersveld national park ...
4       interim interdict spares death  animals hands ...
                              ...                        
7276    two days hurricane ian devastated state florid...
7277                       weather comfortable fair goers
7278    cbs john elliott first alert forecast septembe...
7279    urgent action required support reintroduction ...
7280                         sunshine fried food forecast
Name: DESCRIPTION, Length: 7281, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# pos tagging the words
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_environment6["POSTAG"] = df_environment6["DESCRIPTION"].apply(postag)
df_environment6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0       [(cbs, NN), (vanessa, NN), (murdock, NN), (wea...
1       [(bbc, NN), (weather, NN), (forecaster, NN), (...
2       [(campaigners, NNS), (welcomed, VBD), (doorste...
3       [(worst, JJ), (case, NN), (history, NN), (rich...
4       [(interim, JJ), (interdict, NN), (spares, NNS)...
                              ...                        
7276    [(two, CD), (days, NNS), (hurricane, VBP), (ia...
7277    [(weather, NN), (comfortable, JJ), (fair, JJ),...
7278    [(cbs, NN), (john, NN), (elliott, NN), (first,...
7279    [(urgent, JJ), (action, NN), (required, VBN), ...
7280    [(sunshine, NN), (fried, VBN), (food, NN), (fo...
Name: POSTAG, Length: 7281, dtype: object

In [ ]:
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
# lemmatizing the text
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_environment6["DESCRIPTION"] = df_environment6["DESCRIPTION"].apply(lemmatize_word)
df_environment6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0            cbs vanessa murdock weather forecast april p
1       bbc weather forecaster louise lear say warmth ...
2       campaigners welcome doorstep labour strong hol...
3       worst case history richtersveld national park ...
4       interim interdict spar death animals hand hunt...
                              ...                        
7276    two days hurricane ian devastate state florida...
7277                       weather comfortable fair goers
7278    cbs john elliott first alert forecast september p
7279    urgent action require support reintroduction b...
7280                           sunshine fry food forecast
Name: DESCRIPTION, Length: 7281, dtype: object

In [ ]:
# perform data cleaning on the politics dataset
# display the politics dataset
df_politics.head()

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,01-05-2022 00:50,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,01-05-2022 01:20,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,01-05-2022 01:22,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,01-05-2022 02:05,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,01-05-2022 02:42,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN


In [ ]:
# getting the info
df_politics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60601 entries, 0 to 60600
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TITLE                 60601 non-null  object 
 1   ARTICLE LINK          60601 non-null  object 
 2   PUBLISHED DATE (GMT)  60601 non-null  object 
 3   AUTHOR                35858 non-null  object 
 4   PUBLISHER             60190 non-null  object 
 5   COUNTRY               60600 non-null  object 
 6   CATEGORY              60600 non-null  object 
 7   LANGUAGE              60600 non-null  object 
 8   DESCRIPTION           56219 non-null  object 
 9   IMAGE URL             31496 non-null  object 
 10  VIDEO URL             0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 5.1+ MB


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# converting the data type of the PUBLISHED DATE (GMT) column
df_politics["DATETIME"] = pd.to_datetime(df_politics["PUBLISHED DATE (GMT)"],errors='coerce')
# display the dataframe
df_politics.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,PUBLISHED DATE (GMT),AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,01-05-2022 00:50,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN,2022-01-05 00:50:00
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,01-05-2022 01:20,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 01:20:00
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,01-05-2022 01:22,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN,2022-01-05 01:22:00
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,01-05-2022 02:05,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN,2022-01-05 02:05:00
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,01-05-2022 02:42,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 02:42:00


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the PUBLISHED DATE (GMT) column
df_politics1 = df_politics.drop('PUBLISHED DATE (GMT)', axis=1)
df_politics1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN,2022-01-05 00:50:00
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 01:20:00
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN,2022-01-05 01:22:00
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN,2022-01-05 02:05:00
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 02:42:00


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating date from datetime
df_politics1['DATE'] = df_politics1['DATETIME'].dt.strftime('%d/%m/%Y')
df_politics1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN,2022-01-05 00:50:00,05/01/2022
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 01:20:00,05/01/2022
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN,2022-01-05 01:22:00,05/01/2022
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN,2022-01-05 02:05:00,05/01/2022
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 02:42:00,05/01/2022


In [ ]:
# reference - https://stackoverflow.com/questions/49367877/split-datetime-column-into-separate-date-and-time-columns
# separating time from datetime
df_politics1['GMT'] = df_politics1['DATETIME'].dt.strftime('%H:%M')
df_politics1.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATETIME,DATE,GMT
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN,2022-01-05 00:50:00,05/01/2022,00:50
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 01:20:00,05/01/2022,01:20
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN,2022-01-05 01:22:00,05/01/2022,01:22
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN,2022-01-05 02:05:00,05/01/2022,02:05
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN,2022-01-05 02:42:00,05/01/2022,02:42


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
# drop the DATETIME column
df_politics2 = df_politics1.drop('DATETIME', axis=1)
df_politics2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,Catapulta,Politics,Ireland,Top,English,NaN,NaN,NaN,05/01/2022,00:50
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,NaN,05/01/2022,01:20
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",NaN,NaN,05/01/2022,01:22
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,NaN,NaN,05/01/2022,02:05
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,NaN,05/01/2022,02:42


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_politics2.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE               0
ARTICLE LINK        0
AUTHOR          24743
PUBLISHER         411
COUNTRY             1
CATEGORY            1
LANGUAGE            1
DESCRIPTION      4382
IMAGE URL       29105
VIDEO URL       60601
DATE                1
GMT                 1
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
# replacing the nan values of the AUTHOR,IMAGE URL and VIDEO URL column with Unknown
df_politics2['AUTHOR'] = df_politics2['AUTHOR'].fillna('Unknown')
df_politics2['IMAGE URL'] = df_politics2['IMAGE URL'].fillna('Unknown')
df_politics2['VIDEO URL'] = df_politics2['VIDEO URL'].fillna('Unknown')
df_politics2.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""Should An Garda Siochana become a more divers...",https://politics.ie/threads/should-an-garda-si...,Catapulta,Politics,Ireland,Top,English,NaN,Unknown,Unknown,05/01/2022,00:50
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,01:20
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",Unknown,Unknown,05/01/2022,01:22
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,Unknown,Unknown,05/01/2022,02:05
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,02:42


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# dropping the missing values
df_politics3 = df_politics2.dropna()
df_politics3.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
1,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,01:20
2,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",Unknown,Unknown,05/01/2022,01:22
3,"""RCMP investigation finds incident at Surrey m...",https://vancouversun.com/news/local-news/rcmp-...,Staff Reporter,Vancouver Sun,Canada,World,English,Mounties are working to resolve the matter eit...,Unknown,Unknown,05/01/2022,02:05
4,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,02:42
5,"""Sen. John Cornyn says Russia did no favors du...",https://www.wfaa.com/article/news/politics/ins...,Unknown,Wfaa,United States Of America,Politics,English,The Republican from Texas is also behind a bil...,https://media.tegna-media.com/assets/WFAA/imag...,Unknown,05/01/2022,02:48


In [ ]:
# reference - https://note.nkmk.me/en/python-pandas-nan-judge-count/
# checking for the missing values
df_politics3.isnull().sum()
# note - the code is been adapted from the reference and it is modified

TITLE           0
ARTICLE LINK    0
AUTHOR          0
PUBLISHER       0
COUNTRY         0
CATEGORY        0
LANGUAGE        0
DESCRIPTION     0
IMAGE URL       0
VIDEO URL       0
DATE            0
GMT             0
dtype: int64

In [ ]:
# reference - https://www.geeksforgeeks.org/python-pandas-series-unique/
# getting the unique classes from the category column
print(df_politics3.CATEGORY.unique())
# note - the code is been adapted from the reference and it is modified

['Politics' 'World' 'Top' 'Business' 'Entertainment' 'Technology']


In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/version/0.25/user_guide/indexing.html
# considering only food category
df_politics4 = df_politics3.loc[df_politics3['CATEGORY'] == 'Politics']
df_politics4 = df_politics4.reset_index(drop=True)
df_politics4.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,01:20
1,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",Unknown,Unknown,05/01/2022,01:22
2,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,02:42
3,"""Sen. John Cornyn says Russia did no favors du...",https://www.wfaa.com/article/news/politics/ins...,Unknown,Wfaa,United States Of America,Politics,English,The Republican from Texas is also behind a bil...,https://media.tegna-media.com/assets/WFAA/imag...,Unknown,05/01/2022,02:48
4,"""‘It’s A Great Start,’: Frontline Workers Reac...",https://minnesota.cbslocal.com/2022/04/30/its-...,Unknown,Minnesota Cbs Local,United States Of America,Politics,English,"After months of debate at the capitol, lawmake...",https://minnesota.cbslocal.com/wp-content/uplo...,Unknown,05/01/2022,03:07


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# remove the repeated titles
df_politics4.drop_duplicates(subset="TITLE",keep=False, inplace=True)
df_politics5 = df_politics4.reset_index(drop=True)
df_politics5.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,01:20
1,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",Unknown,Unknown,05/01/2022,01:22
2,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,02:42
3,"""Sen. John Cornyn says Russia did no favors du...",https://www.wfaa.com/article/news/politics/ins...,Unknown,Wfaa,United States Of America,Politics,English,The Republican from Texas is also behind a bil...,https://media.tegna-media.com/assets/WFAA/imag...,Unknown,05/01/2022,02:48
4,"""‘It’s A Great Start,’: Frontline Workers Reac...",https://minnesota.cbslocal.com/2022/04/30/its-...,Unknown,Minnesota Cbs Local,United States Of America,Politics,English,"After months of debate at the capitol, lawmake...",https://minnesota.cbslocal.com/wp-content/uplo...,Unknown,05/01/2022,03:07


In [ ]:
# remove emoticons from the entire dataset
# reference - https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
df_politics6 = df_politics5.astype(str).apply(lambda z: z.str.encode('ascii', 'ignore').str.decode('ascii'))
df_politics6.head()
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT
0,"""No rush on Russia G20 decision: Morrison""",https://thewest.com.au/politics/no-rush-on-rus...,Paul Osborne,The West,Australia,Politics,English,There is no urgency on a decision as to how to...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,01:20
1,"""Kim Kardashian, Pete Davidson and More Stars ...",https://www.hollywoodreporter.com/news/politic...,Ryan Gajewski,The Hollywood Reporter,United States Of America,Politics,English,"In a return to some normalcy, journalists, gov...",Unknown,Unknown,05/01/2022,01:22
2,"""I'm a local MP first, says Frydenberg""",https://thewest.com.au/politics/im-a-local-mp-...,Paul Osborne,The West,Australia,Politics,English,Treasurer Josh Frydenberg faces a tough battle...,https://images.thewest.com.au/publication/C-66...,Unknown,05/01/2022,02:42
3,"""Sen. John Cornyn says Russia did no favors du...",https://www.wfaa.com/article/news/politics/ins...,Unknown,Wfaa,United States Of America,Politics,English,The Republican from Texas is also behind a bil...,https://media.tegna-media.com/assets/WFAA/imag...,Unknown,05/01/2022,02:48
4,"""Its A Great Start,: Frontline Workers React T...",https://minnesota.cbslocal.com/2022/04/30/its-...,Unknown,Minnesota Cbs Local,United States Of America,Politics,English,"After months of debate at the capitol, lawmake...",https://minnesota.cbslocal.com/wp-content/uplo...,Unknown,05/01/2022,03:07


In [ ]:
# converting the text into lower case
# reference - https://stackoverflow.com/questions/67897023/how-do-i-define-a-function-that-returns-my-string-in-lower-case
def lower_text(text):
    text = text.lower()
    return text
df_politics6["DESCRIPTION"] = df_politics6["DESCRIPTION"].apply(lower_text)
df_politics6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        there is no urgency on a decision as to how to...
1        in a return to some normalcy, journalists, gov...
2        treasurer josh frydenberg faces a tough battle...
3        the republican from texas is also behind a bil...
4        after months of debate at the capitol, lawmake...
                               ...                        
37726    new zealand condemns unequivocally russias att...
37727    leke baiyewu and abiodun sanusi x-ray the poli...
37728    apparently, it's a no-go given what we know ab...
37729    a former governorship aspirant on the platform...
37730    the labour party presidential candidate, peter...
Name: DESCRIPTION, Length: 37731, dtype: object

In [ ]:
# printing the punctuations
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# removing the punctuations
# reference - https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
def remove_punct(text):
    for char in string.punctuation:
        text = text.replace(char, ' ')
    return text
df_politics6["DESCRIPTION"] = df_politics6["DESCRIPTION"].apply(remove_punct)
df_politics6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        there is no urgency on a decision as to how to...
1        in a return to some normalcy  journalists  gov...
2        treasurer josh frydenberg faces a tough battle...
3        the republican from texas is also behind a bil...
4        after months of debate at the capitol  lawmake...
                               ...                        
37726    new zealand condemns unequivocally russias att...
37727    leke baiyewu and abiodun sanusi x ray the poli...
37728    apparently  it s a no go given what we know ab...
37729    a former governorship aspirant on the platform...
37730    the labour party presidential candidate  peter...
Name: DESCRIPTION, Length: 37731, dtype: object

In [ ]:
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = " ".join([z for z in word_tokens if z not in stop_words])
    return text
df_politics6["DESCRIPTION"] = df_politics6["DESCRIPTION"].apply(remove_stop)
df_politics6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        urgency decision respond russian president vla...
1        return normalcy journalists government officia...
2        treasurer josh frydenberg faces tough battle i...
3        republican texas also behind bill make easier ...
4        months debate capitol lawmakers came together ...
                               ...                        
37726    new zealand condemns unequivocally russias att...
37727    leke baiyewu abiodun sanusi x ray political hi...
37728    apparently go given know ongoing investigation...
37729    former governorship aspirant platform peoples ...
37730    labour party presidential candidate peter obi ...
Name: DESCRIPTION, Length: 37731, dtype: object

In [ ]:
# remove the numbers and words containing number
# reference - https://www.youtube.com/watch?v=hZ59DYYgPRE
def no_number(text):
    text = "".join([z for z in text if not z.isdigit()])
    return text
df_politics6["DESCRIPTION"] = df_politics6["DESCRIPTION"].apply(no_number)
df_politics6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        urgency decision respond russian president vla...
1        return normalcy journalists government officia...
2        treasurer josh frydenberg faces tough battle i...
3        republican texas also behind bill make easier ...
4        months debate capitol lawmakers came together ...
                               ...                        
37726    new zealand condemns unequivocally russias att...
37727    leke baiyewu abiodun sanusi x ray political hi...
37728    apparently go given know ongoing investigation...
37729    former governorship aspirant platform peoples ...
37730    labour party presidential candidate peter obi ...
Name: DESCRIPTION, Length: 37731, dtype: object

In [ ]:
# pos tagging the words
# reference - https://michael-fuchs-python.netlify.app/2021/05/25/nlp-text-pre-processing-ii-tokenization-and-stop-words/
stop_words = set(stopwords.words('english'))
def postag (text):
    word_tokens = word_tokenize(text)
    text = nltk.pos_tag([z for z in word_tokens if not z in stop_words])
    return text
df_politics6["POSTAG"] = df_politics6["DESCRIPTION"].apply(postag)
df_politics6["POSTAG"]
# note - the code is been adapted from the reference and it is modified

0        [(urgency, JJ), (decision, NN), (respond, NN),...
1        [(return, NN), (normalcy, JJ), (journalists, N...
2        [(treasurer, NN), (josh, NN), (frydenberg, NN)...
3        [(republican, JJ), (texas, NN), (also, RB), (b...
4        [(months, NNS), (debate, NN), (capitol, NN), (...
                               ...                        
37726    [(new, JJ), (zealand, NNP), (condemns, NN), (u...
37727    [(leke, NN), (baiyewu, NN), (abiodun, NN), (sa...
37728    [(apparently, RB), (go, VB), (given, VBN), (kn...
37729    [(former, JJ), (governorship, NN), (aspirant, ...
37730    [(labour, JJ), (party, NN), (presidential, JJ)...
Name: POSTAG, Length: 37731, dtype: object

In [ ]:
# lemmatizing the text
# reference - https://marcobonzanini.com/2015/01/26/stemming-lemmatisation-and-pos-tagging-with-python-and-nltk/
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    text = " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])
    return text
df_politics6["DESCRIPTION"] = df_politics6["DESCRIPTION"].apply(lemmatize_word)
df_politics6["DESCRIPTION"]
# note - the code is been adapted from the reference and it is modified

0        urgency decision respond russian president vla...
1        return normalcy journalists government officia...
2        treasurer josh frydenberg face tough battle in...
3        republican texas also behind bill make easier ...
4        months debate capitol lawmakers come together ...
                               ...                        
37726    new zealand condemn unequivocally russias atte...
37727    leke baiyewu abiodun sanusi x ray political hi...
37728    apparently go give know ongoing investigations...
37729    former governorship aspirant platform people d...
37730    labour party presidential candidate peter obi ...
Name: DESCRIPTION, Length: 37731, dtype: object

In [ ]:
# reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
# combined all the dataframes and make it one dataframe
df_news_com = [df_food6, df_sports6, df_health6,df_business6,df_entertainment6,df_environment6,df_politics6]
df_news_com1 = pd.concat(df_news_com)
df_news_com1
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT,POSTAG
0,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,business duo create home hair kit bridge gap b...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16,"[(business, NN), (duo, NN), (created, VBD), (h..."
1,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,definitely need love make confident secure hap...,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25,"[(definitely, RB), (need, MD), (love, VB), (ma..."
2,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,class session view entire post,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25,"[(class, NN), (session, NN), (view, NN), (enti..."
3,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,ultra chic contemporary home lap pool tennis c...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53,"[(ultra, JJ), (chic, JJ), (contemporary, JJ), ..."
4,"""Viral Tiktok claims Sunnyboys were back at Wo...",https://www.dailymail.co.uk/femail/food/articl...,Unknown,Mail Online,United Kingdom,Food,English,australian shoppers send spin tiktok clip go v...,https://i.dailymail.co.uk/1s/2022/05/01/02/572...,Unknown,05/01/2022,01:32,"[(australian, JJ), (shoppers, NNS), (sent, VBD..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37726,"""'New Zealand condemns Russias annexation atte...",http://www.voxy.co.nz/politics/5/407843,Unknown,Voxy,New Zealand,Politics,English,new zealand condemn unequivocally russias atte...,Unknown,Unknown,30/09/2022,23:02,"[(new, JJ), (zealand, NNP), (condemns, NN), (u..."
37727,"""Nigeria @ 62: So far, not so good""",https://punchng.com/nigeria-62-so-far-not-so-g...,Our Reporters,Punch,Nigeria,Politics,English,leke baiyewu abiodun sanusi x ray political hi...,Unknown,Unknown,30/09/2022,23:25,"[(leke, NN), (baiyewu, NN), (abiodun, NN), (sa..."
37728,"""Most Voters Say Trump Should Not Be Allowed T...",https://www.huffpost.com/entry/voters-donald-t...,Unknown,Huffpost,United States Of America,Politics,English,apparently go give know ongoing investigations...,Unknown,Unknown,30/09/2022,23:25,"[(apparently, RB), (go, VB), (given, VBN), (kn..."
37729,"""Babayemi congratulates Adeleke on Supreme Cou...",https://punchng.com/babayemi-congratulates-ade...,Bola Bamigbola,Punch,Nigeria,Politics,English,former governorship aspirant platform people d...,Unknown,Unknown,30/09/2022,23:44,"[(former, JJ), (governorship, NN), (aspirant, ..."


In [ ]:
# reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
# resetting the index of the combined data frame
df_news_com2 = df_news_com1.reset_index(drop=True)
df_news_com2
# note - the code is been adapted from the reference and it is modified

,TITLE,ARTICLE LINK,AUTHOR,PUBLISHER,COUNTRY,CATEGORY,LANGUAGE,DESCRIPTION,IMAGE URL,VIDEO URL,DATE,GMT,POSTAG
0,"""Business duo turn over $70K a month with thei...",https://www.dailymail.co.uk/femail/article-107...,Unknown,Mail Online,United Kingdom,Food,English,business duo create home hair kit bridge gap b...,https://i.dailymail.co.uk/1s/2022/04/29/05/571...,Unknown,05/01/2022,00:16,"[(business, NN), (duo, NN), (created, VBD), (h..."
1,"""People Are Sharing ""Impulsive And Expensive"" ...",https://www.buzzfeed.com/fabianabuontempo/peop...,Unknown,Buzzfeed,United Kingdom,Food,English,definitely need love make confident secure hap...,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25,"[(definitely, RB), (need, MD), (love, VB), (ma..."
2,"""Why You Shouldn't Butter The Pan When Making ...",https://www.buzzfeed.com/rossyoder/culinary-sc...,Unknown,Buzzfeed,United Kingdom,Food,English,class session view entire post,https://img.buzzfeed.com/buzzfeed-static/stati...,Unknown,05/01/2022,00:25,"[(class, NN), (session, NN), (view, NN), (enti..."
3,"""Quaint beachside cottage on the Mornington Pe...",https://www.dailymail.co.uk/femail/homes/artic...,Unknown,Mail Online,United Kingdom,Food,English,ultra chic contemporary home lap pool tennis c...,https://i.dailymail.co.uk/1s/2022/04/29/07/571...,Unknown,05/01/2022,00:53,"[(ultra, JJ), (chic, JJ), (contemporary, JJ), ..."
4,"""Viral Tiktok claims Sunnyboys were back at Wo...",https://www.dailymail.co.uk/femail/food/articl...,Unknown,Mail Online,United Kingdom,Food,English,australian shoppers send spin tiktok clip go v...,https://i.dailymail.co.uk/1s/2022/05/01/02/572...,Unknown,05/01/2022,01:32,"[(australian, JJ), (shoppers, NNS), (sent, VBD..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
823100,"""'New Zealand condemns Russias annexation atte...",http://www.voxy.co.nz/politics/5/407843,Unknown,Voxy,New Zealand,Politics,English,new zealand condemn unequivocally russias atte...,Unknown,Unknown,30/09/2022,23:02,"[(new, JJ), (zealand, NNP), (condemns, NN), (u..."
823101,"""Nigeria @ 62: So far, not so good""",https://punchng.com/nigeria-62-so-far-not-so-g...,Our Reporters,Punch,Nigeria,Politics,English,leke baiyewu abiodun sanusi x ray political hi...,Unknown,Unknown,30/09/2022,23:25,"[(leke, NN), (baiyewu, NN), (abiodun, NN), (sa..."
823102,"""Most Voters Say Trump Should Not Be Allowed T...",https://www.huffpost.com/entry/voters-donald-t...,Unknown,Huffpost,United States Of America,Politics,English,apparently go give know ongoing investigations...,Unknown,Unknown,30/09/2022,23:25,"[(apparently, RB), (go, VB), (given, VBN), (kn..."
823103,"""Babayemi congratulates Adeleke on Supreme Cou...",https://punchng.com/babayemi-congratulates-ade...,Bola Bamigbola,Punch,Nigeria,Politics,English,former governorship aspirant platform people d...,Unknown,Unknown,30/09/2022,23:44,"[(former, JJ), (governorship, NN), (aspirant, ..."


In [ ]:
# reference - https://cmdlinetips.com/2020/05/how-to-save-pandas-dataframe-as-gzip-zip-file/
# converting the combined dataframe to the zip file which is required to be imported in colab
df_news_com2.to_csv("df_news_combined.csv.zip", index=False, compression="zip")
# note - the code is been adapted from the reference and it is modified

In [ ]:
# reference - https://towardsdatascience.com/how-to-export-pandas-dataframe-to-csv-2038e43d9c03
df_news_com2.to_csv('df_news_combined.csv', index=False)
# note - the code is been adapted from the reference and it is modified